# Deep Learning workflow
1. Load the data
2. Process the data
3. Build a neural network model
4. Define a loss function
5. Define the Optimozer
6. Train the model
7. Evaluate the model
8. Save the model for future use






# New Section

In [ ]:
import torch
from torch import nn                                 # Neural Network
from torch.utils.data import DataLoader              # DataLoader
from torchvision import datasets                     # torchvision library for specially for the computer vision task
from torchvision.transforms import ToTensor          # Convert image pixel values [0, 255] from the integer ranges [0.0, 1.0].

PyTorch offers domain-specific libraries such as TorchText, TorchVision, and TorchAudio, all of which include datasets.

The torchvision.datasets module contains Dataset objects for many real-world vision data like FashionMNIST, CIFAR, COCO.

In this tutorial, we use the FashionMNIST dataset. Every TorchVision Dataset includes two arguments: transform and target_transform to modify the samples and labels respectively.

In [ ]:
# Download training data from open datasets.

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

100%|██████████| 26.4M/26.4M [00:01<00:00, 16.8MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 306kB/s]
100%|██████████| 4.42M/4.42M [00:00<00:00, 5.04MB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 19.6MB/s]


We pass the Dataset as an argument to DataLoader.

This wraps an iterable over our dataset, and supports automatic batching, sampling, shuffling and multiprocess data loading.

Here we define a batch size of 64, i.e. each element in the dataloader iterable will return a batch of 64 features and labels.

In [ ]:
batch_size = 64        # model take a 64 sample at a time

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")      # where N: Batch size, C = 1 : Grayscale (Black & White), H: Image Height, W: image Weights
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


# Creating Models
To define a neural network in PyTorch, we create a class that inherits from nn.Module.

We define the layers of the network in the __init__ function and specify how data will pass through the network in the forward function.


In [ ]:
# Initialization
device = "cuda" if torch.cuda.is_available() else "cpu"

# Define model
import torch.nn as nn

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()

        # Defined Layers
        self.fc1 = nn.Linear(28*28, 512)
        self.r1 = nn.ReLU()

        self.fc2 = nn.Linear(512, 512)
        self.r2 = nn.ReLU()

        self.fc3 = nn.Linear(512, 512)
        self.r3 = nn.ReLU()

        self.fc4 = nn.Linear(512, 10)      # Output layer (10 classes)

    def forward(self, x):
        x = self.flatten(x)

        # Layer 1
        x = self.fc1(x)
        x = self.r1(x)

        # Layer 2
        x = self.fc2(x)
        x = self.r2(x)

        # Layer 3
        x = self.fc3(x)
        x = self.r3(x)

        # Layer 4
        logits = self.fc4(x)

        return logits

# Initialize

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=784, out_features=512, bias=True)
  (r1): ReLU()
  (fc2): Linear(in_features=512, out_features=512, bias=True)
  (r2): ReLU()
  (fc3): Linear(in_features=512, out_features=512, bias=True)
  (r3): ReLU()
  (fc4): Linear(in_features=512, out_features=10, bias=True)
)


# Optimizing the Model Parameters
To train a model, we need a loss function and an optimizer.

In [ ]:
loss_fn = nn.CrossEntropyLoss()                             # CEL: loss Function for multi-class classification, BCE: Binary classifications
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)    # SGD : Optimizer, lr: Learning_rate

In a single training loop, the model makes predictions on the training dataset (fed to it in batches), and backpropagates the prediction error to adjust the model's parameters.

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()                             # Gradient must zero, reset every iteration

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

# We also check the model's performance against the test dataset to ensure it is learning.

In [ ]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

The training process is conducted over several iterations (epochs).

During each epoch, the model learns parameters to make better predictions.

We print the model's accuracy and loss at each epoch; we'd like to see the accuracy increase and the loss decrease with every epoch.

In [ ]:
epochs = 5                        #  One full pass over the dataset
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.301458  [   64/60000]
loss: 2.298745  [ 6464/60000]
loss: 2.295789  [12864/60000]
loss: 2.294936  [19264/60000]
loss: 2.283821  [25664/60000]
loss: 2.288132  [32064/60000]
loss: 2.284812  [38464/60000]
loss: 2.280998  [44864/60000]
loss: 2.278941  [51264/60000]
loss: 2.275712  [57664/60000]
Test Error: 
 Accuracy: 35.7%, Avg loss: 2.272278 

Epoch 2
-------------------------------
loss: 2.274998  [   64/60000]
loss: 2.273387  [ 6464/60000]
loss: 2.262054  [12864/60000]
loss: 2.266813  [19264/60000]
loss: 2.251383  [25664/60000]
loss: 2.243963  [32064/60000]
loss: 2.253301  [38464/60000]
loss: 2.238846  [44864/60000]
loss: 2.239194  [51264/60000]
loss: 2.226059  [57664/60000]
Test Error: 
 Accuracy: 35.3%, Avg loss: 2.224423 

Epoch 3
-------------------------------
loss: 2.231255  [   64/60000]
loss: 2.229787  [ 6464/60000]
loss: 2.203721  [12864/60000]
loss: 2.214800  [19264/60000]
loss: 2.189648  [25664/60000]
loss: 2.161458  [32064/600

# Saving Models
A common way to save a model is to serialize the internal state dictionary (containing the model parameters).

In [ ]:
torch.save(model.state_dict(), "model.pth")           # Save weights
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


# Save Entire Model

# Loading Models
The process for loading a model includes re-creating the model structure and loading the state dictionary into it.

In [ ]:
torch.save(model,'model.pth')                           # Saves architecture + weights
print("Save Pytorch entire Model State to model.pth")

Save Pytorch entire Model State to model.pth


In [ ]:
model = torch.load("model.pth", map_location=device, weights_only=False)      # Load entire model
model.eval()

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=784, out_features=512, bias=True)
  (r1): ReLU()
  (fc2): Linear(in_features=512, out_features=512, bias=True)
  (r2): ReLU()
  (fc3): Linear(in_features=512, out_features=512, bias=True)
  (r3): ReLU()
  (fc4): Linear(in_features=512, out_features=10, bias=True)
)

This model can now be used to make predictions.

In [ ]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
